In [40]:
###i dati contenuti nel dataset sono, come da informazioni rilevate dai documenti, già stae in parte preprocessate, ed ogni variabile normalizzata in un range da -1 a +1, questo semplifica notevolmente il lavoro


In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
# from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, \
    accuracy_score, precision_score, recall_score
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import RFE, RFECV, SelectKBest, f_classif, mutual_info_classif
from sklearn.decomposition import PCA
from sklearn.manifold import MDS, TSNE, Isomap
from sklearn import tree

from itertools import accumulate
import pickle


import warnings # ignore FutureWarning TSNE
warnings.filterwarnings("ignore")

In [42]:

features = list()
with open('features.txt') as f:
    features = [line.split()[1] for line in f.readlines()]
print('No of Features: {}'.format(len(features)))


No of Features: 561


In [43]:
X_train = pd.read_csv('X_train.txt', delim_whitespace=True, header=None, encoding='latin-1')
X_train.columns = features

# add subject column to the dataframe
X_train['subject'] = pd.read_csv('subject_train.txt', header=None, squeeze=True)

y_train = pd.read_csv('y_train.txt', names=['Activity'], squeeze=True)
y_train_labels = y_train.map({1: 'WALKING', 2:'WALKING_UPSTAIRS',3:'WALKING_DOWNSTAIRS',\
                       4:'SITTING', 5:'STANDING',6:'LAYING'})

# put all columns in a single dataframe
train = X_train
train['Activity'] = y_train
train['ActivityName'] = y_train_labels

In [44]:
X_train

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity,ActivityName
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,5,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,5,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,5,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,5,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,5,STANDING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,0.299665,-0.057193,-0.181233,-0.195387,0.039905,0.077078,-0.282301,0.043616,0.060410,0.210795,...,-0.190437,0.829718,0.206972,-0.425619,-0.791883,0.238604,0.049819,30,2,WALKING_UPSTAIRS
7348,0.273853,-0.007749,-0.147468,-0.235309,0.004816,0.059280,-0.322552,-0.029456,0.080585,0.117440,...,0.064907,0.875679,-0.879033,0.400219,-0.771840,0.252676,0.050053,30,2,WALKING_UPSTAIRS
7349,0.273387,-0.017011,-0.045022,-0.218218,-0.103822,0.274533,-0.304515,-0.098913,0.332584,0.043999,...,0.052806,-0.266724,0.864404,0.701169,-0.779133,0.249145,0.040811,30,2,WALKING_UPSTAIRS
7350,0.289654,-0.018843,-0.158281,-0.219139,-0.111412,0.268893,-0.310487,-0.068200,0.319473,0.101702,...,-0.101360,0.700740,0.936674,-0.589479,-0.785181,0.246432,0.025339,30,2,WALKING_UPSTAIRS


In [45]:
X_train.min(axis = 1).min(), X_train.max(axis = 1).max()

(-1.0, 30.0)

In [46]:
X_train.shape

(7352, 564)

In [47]:
##The random sampling was conducteded on participants, not on the entire dataset (records). Therefore, data in train and test set refer to different persons. This means that in the classification tasks we have to suppose to predict the activity performed by an unknown individual looking at the data generated by his smartphone.

In [48]:
X_trainc = X_train.drop(['subject', 'Activity', 'ActivityName'], axis= 1)

In [49]:
X_trainc

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,0.414503,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,0.404573,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.087753,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,0.299665,-0.057193,-0.181233,-0.195387,0.039905,0.077078,-0.282301,0.043616,0.060410,0.210795,...,-0.070157,-0.588433,-0.880324,-0.190437,0.829718,0.206972,-0.425619,-0.791883,0.238604,0.049819
7348,0.273853,-0.007749,-0.147468,-0.235309,0.004816,0.059280,-0.322552,-0.029456,0.080585,0.117440,...,0.165259,-0.390738,-0.680744,0.064907,0.875679,-0.879033,0.400219,-0.771840,0.252676,0.050053
7349,0.273387,-0.017011,-0.045022,-0.218218,-0.103822,0.274533,-0.304515,-0.098913,0.332584,0.043999,...,0.195034,0.025145,-0.304029,0.052806,-0.266724,0.864404,0.701169,-0.779133,0.249145,0.040811
7350,0.289654,-0.018843,-0.158281,-0.219139,-0.111412,0.268893,-0.310487,-0.068200,0.319473,0.101702,...,0.013865,0.063907,-0.344314,-0.101360,0.700740,0.936674,-0.589479,-0.785181,0.246432,0.025339


In [50]:
import pandas as pd
import numpy as np

def drop_highly_correlated_columns(df, threshold=0.85):
    # Calculate the correlation matrix
    corr_matrix = df.corr().abs()

    # Select upper triangular of correlation matrix (excluding the diagonal)
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    # Find highly correlated columns and drop one from each pair
    columns_to_drop = [column for column in upper_tri.columns if any(upper_tri[column] >= threshold)]
    df = df.drop(columns=columns_to_drop)

    return df

# Example usage:
# Assuming you have a DataFrame called 'X_trainc'
# After calling the function, you will get the DataFrame with highly correlated columns removed.
X_trainc = drop_highly_correlated_columns(X_trainc, threshold=0.85)

In [51]:
X_trainc

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-entropy()-X,tBodyAcc-entropy()-Z,"tBodyAcc-arCoeff()-X,1","tBodyAcc-arCoeff()-X,3","tBodyAcc-arCoeff()-X,4","tBodyAcc-arCoeff()-Y,1",...,fBodyBodyGyroMag-meanFreq(),fBodyBodyGyroMag-skewness(),fBodyBodyGyroJerkMag-min(),fBodyBodyGyroJerkMag-maxInds,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)"
0,0.288585,-0.020294,-0.132905,-0.995279,-0.407747,-0.602122,0.929294,0.359910,-0.058526,0.256892,...,-0.128989,0.586156,-0.991048,-1.000000,-0.074323,-0.298676,-0.112754,0.030400,-0.464761,-0.018446
1,0.278419,-0.016411,-0.123520,-0.998245,-0.714892,-0.570979,0.611627,0.284213,0.284595,0.115705,...,-0.271958,-0.336310,-0.994440,-1.000000,0.158075,-0.595051,0.053477,-0.007435,-0.732626,0.703511
2,0.279653,-0.019467,-0.113462,-0.995380,-0.592235,-0.570979,0.273025,0.337202,-0.164739,0.017150,...,-0.212728,-0.535352,-0.995866,-0.555556,0.414503,-0.390748,-0.118559,0.177899,0.100699,0.808529
3,0.279174,-0.026201,-0.123283,-0.996091,-0.627446,-0.911872,0.061436,0.198204,-0.264307,0.072545,...,-0.035684,-0.230091,-0.995732,-0.936508,0.404573,-0.117290,-0.036788,-0.012892,0.640011,-0.485366
4,0.276629,-0.016570,-0.115362,-0.998139,-0.786553,-0.761434,0.313276,0.191161,0.086904,0.257615,...,-0.273582,-0.510282,-0.997418,-0.936508,0.087753,-0.351471,0.123320,0.122542,0.693578,-0.615971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,0.299665,-0.057193,-0.181233,-0.195387,0.195643,-0.082077,-0.561607,-0.078255,-0.056751,-0.094106,...,-0.376234,-0.170845,-0.925241,-0.904762,-0.070157,-0.588433,-0.190437,0.829718,0.206972,-0.425619
7348,0.273853,-0.007749,-0.147468,-0.235309,0.178138,-0.051532,-0.361660,0.206839,-0.154722,0.032725,...,-0.296176,0.019626,-0.761880,-0.904762,0.165259,-0.390738,0.064907,0.875679,-0.879033,0.400219
7349,0.273387,-0.017011,-0.045022,-0.218218,0.173131,0.152003,-0.376773,0.063584,-0.017019,-0.004323,...,-0.320249,0.490169,-0.692582,-0.904762,0.195034,0.025145,0.052806,-0.266724,0.864404,0.701169
7350,0.289654,-0.018843,-0.158281,-0.219139,0.160333,0.021678,-0.475165,0.009588,-0.038354,-0.277801,...,-0.412332,0.308105,-0.886154,-0.904762,0.013865,0.063907,-0.101360,0.700740,0.936674,-0.589479


In [52]:
X_trainc.var().describe()

count    152.000000
mean       0.086685
std        0.081885
min        0.001665
25%        0.042112
50%        0.063933
75%        0.103484
max        0.500873
dtype: float64

In [53]:
X_trainc['Activity'] = y_train
X_trainc['ActivityName'] = y_train_labels

In [54]:
X_trainc['subject'] = pd.read_csv('subject_train.txt', header=None, squeeze=True)

In [55]:
X_trainc

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-entropy()-X,tBodyAcc-entropy()-Z,"tBodyAcc-arCoeff()-X,1","tBodyAcc-arCoeff()-X,3","tBodyAcc-arCoeff()-X,4","tBodyAcc-arCoeff()-Y,1",...,fBodyBodyGyroJerkMag-maxInds,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)",Activity,ActivityName,subject
0,0.288585,-0.020294,-0.132905,-0.995279,-0.407747,-0.602122,0.929294,0.359910,-0.058526,0.256892,...,-1.000000,-0.074323,-0.298676,-0.112754,0.030400,-0.464761,-0.018446,5,STANDING,1
1,0.278419,-0.016411,-0.123520,-0.998245,-0.714892,-0.570979,0.611627,0.284213,0.284595,0.115705,...,-1.000000,0.158075,-0.595051,0.053477,-0.007435,-0.732626,0.703511,5,STANDING,1
2,0.279653,-0.019467,-0.113462,-0.995380,-0.592235,-0.570979,0.273025,0.337202,-0.164739,0.017150,...,-0.555556,0.414503,-0.390748,-0.118559,0.177899,0.100699,0.808529,5,STANDING,1
3,0.279174,-0.026201,-0.123283,-0.996091,-0.627446,-0.911872,0.061436,0.198204,-0.264307,0.072545,...,-0.936508,0.404573,-0.117290,-0.036788,-0.012892,0.640011,-0.485366,5,STANDING,1
4,0.276629,-0.016570,-0.115362,-0.998139,-0.786553,-0.761434,0.313276,0.191161,0.086904,0.257615,...,-0.936508,0.087753,-0.351471,0.123320,0.122542,0.693578,-0.615971,5,STANDING,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,0.299665,-0.057193,-0.181233,-0.195387,0.195643,-0.082077,-0.561607,-0.078255,-0.056751,-0.094106,...,-0.904762,-0.070157,-0.588433,-0.190437,0.829718,0.206972,-0.425619,2,WALKING_UPSTAIRS,30
7348,0.273853,-0.007749,-0.147468,-0.235309,0.178138,-0.051532,-0.361660,0.206839,-0.154722,0.032725,...,-0.904762,0.165259,-0.390738,0.064907,0.875679,-0.879033,0.400219,2,WALKING_UPSTAIRS,30
7349,0.273387,-0.017011,-0.045022,-0.218218,0.173131,0.152003,-0.376773,0.063584,-0.017019,-0.004323,...,-0.904762,0.195034,0.025145,0.052806,-0.266724,0.864404,0.701169,2,WALKING_UPSTAIRS,30
7350,0.289654,-0.018843,-0.158281,-0.219139,0.160333,0.021678,-0.475165,0.009588,-0.038354,-0.277801,...,-0.904762,0.013865,0.063907,-0.101360,0.700740,0.936674,-0.589479,2,WALKING_UPSTAIRS,30


In [56]:
#X_trainc.to_csv('X_trainc.csv', index=False)

In [57]:
X_test = pd.read_csv('X_test.txt', delim_whitespace=True, header=None, encoding='latin-1')
X_test.columns = features

# add subject column to the dataframe
X_test['subject'] = pd.read_csv('subject_test.txt', header=None, squeeze=True)

y_test = pd.read_csv('y_test.txt', names=['Activity'], squeeze=True)
y_test_labels = y_test.map({1: 'WALKING', 2:'WALKING_UPSTAIRS',3:'WALKING_DOWNSTAIRS',\
                       4:'SITTING', 5:'STANDING',6:'LAYING'})

# put all columns in a single dataframe
test = X_test
test['Activity'] = y_test
test['ActivityName'] = y_test_labels

In [58]:
type(y_test)

pandas.core.series.Series

In [59]:
col = X_trainc.columns

In [60]:
X_testc = test[col]

In [61]:
#X_testc.to_csv('X_testc.csv', index=False)

In [62]:
y_train

0       5
1       5
2       5
3       5
4       5
       ..
7347    2
7348    2
7349    2
7350    2
7351    2
Name: Activity, Length: 7352, dtype: int64